In [1]:
import os
from pyspark import SparkContext
sc = SparkContext(appName="search", master=os.environ['MASTER'])
import pysam

In [2]:
from pyspark.sql import SQLContext, Row
sqlCtx = SQLContext(sc)

In [30]:
#read bam file and put identicalID, tlen and read seq into sparkContext
filename = "../../../GenomeData/NA21144.chrom20.ILLUMINA.bwa.GIH.low_coverage.20130415.bam"
samfile = pysam.AlignmentFile(filename, 'rb')
bf = sc.parallelize([' '])
tlenList = []
for ite in samfile.fetch():
    if abs(ite.tlen) > 1000:
        tlenList.append(("NA21144", ite.tlen, ite.seq))

lines = sc.parallelize(tlenList)  

#populate data into sqlContext
seqVar = lines.map(lambda p: Row(iid=p[0], tlen=p[1], seq=p[2]))
seqVarTable = sqlCtx.inferSchema(seqVar)
seqVarTable.registerAsTable("seqVar")



#seqVarTable.printSchema()
#print seqVarTable.collect()
#seqList = sqlCtx.sql("SELECT seq from seqVar")
